In [ ]:
!pip install flask flask-cors pyngrok tensorflow pillow numpy joblib --quiet

In [ ]:
from pyngrok import ngrok

# Set your auth token
ngrok.set_auth_token("please paste you ngrok authtoken here ")


In [ ]:
# ==========================================
# AI Flood Relief - Combined Backend (Citizen + Admin + Relief)
# ==========================================

!pip install flask flask-cors pyngrok tensorflow pillow numpy joblib --quiet

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading, sqlite3, os, numpy as np, tensorflow as tf, joblib
from PIL import Image
import io
from google.colab import drive

# -------------------------------
# 1️⃣ Mount Drive & Paths
# -------------------------------
drive.mount('/content/drive', force_remount=True)
BASE_DIR = "/content/drive/My Drive/AI_Flood"
os.makedirs(BASE_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "reports.db")
MODEL_PATH_H5 = os.path.join(BASE_DIR, "flood.h5")       # CNN model for citizen images
MODEL_PATH_PKL = os.path.join(BASE_DIR, "flood_model.pkl") # Tabular ML model for admin

# -------------------------------
# 2️⃣ Stop old processes
# -------------------------------
!pkill -f ngrok
!fuser -k 5000/tcp || true

# -------------------------------
# 3️⃣ Load models
# -------------------------------
# 3a. Citizen CNN model
from tensorflow.keras.layers import (
    Conv2D, MaxPool2D, UpSampling2D,
    Dropout, BatchNormalization, Add,
    Multiply, concatenate, Layer
)

class EncoderBlock(Layer):
    def __init__(self, filters, rate, pooling=True, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.rate = rate
        self.pooling = pooling
        self.c1 = Conv2D(filters, 3, padding="same", activation="relu")
        self.drop = Dropout(rate)
        self.c2 = Conv2D(filters, 3, padding="same", activation="relu")
        self.pool = MaxPool2D()

    def call(self, x):
        x = self.c1(x)
        x = self.drop(x)
        x = self.c2(x)
        return (self.pool(x), x) if self.pooling else x

    def get_config(self):
        return {"filters": self.filters, "rate": self.rate, "pooling": self.pooling}

class DecoderBlock(Layer):
    def __init__(self, filters, rate, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.rate = rate
        self.up = UpSampling2D()
        self.enc = EncoderBlock(filters, rate, pooling=False)

    def call(self, x):
        x, skip = x
        x = self.up(x)
        x = concatenate([x, skip])
        return self.enc(x)

    def get_config(self):
        return {"filters": self.filters, "rate": self.rate}

class AttentionGate(Layer):
    def __init__(self, filters, bn=True, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.bn = bn
        self.conv = Conv2D(filters, 3, padding="same", activation="relu")
        self.down = Conv2D(filters, 3, strides=2, padding="same", activation="relu")
        self.learn = Conv2D(1, 1, activation="sigmoid")
        self.up = UpSampling2D()
        self.bn_layer = BatchNormalization()

    def call(self, x):
        g, skip = x
        x = Add()([self.conv(g), self.down(skip)])
        x = self.learn(x)
        x = self.up(x)
        x = Multiply()([x, skip])
        return self.bn_layer(x) if self.bn else x

    def get_config(self):
        return {"filters": self.filters, "bn": self.bn}

# Load CNN model
if not os.path.exists(MODEL_PATH_H5):
    raise FileNotFoundError("❌ Citizen CNN model missing")

cnn_model = tf.keras.models.load_model(
    MODEL_PATH_H5,
    custom_objects={"EncoderBlock": EncoderBlock, "DecoderBlock": DecoderBlock, "AttentionGate": AttentionGate},
    compile=False
)
print("✅ CNN model loaded")

# Load Admin ML model
if not os.path.exists(MODEL_PATH_PKL):
    raise FileNotFoundError("❌ Admin ML model missing")
ml_model = joblib.load(MODEL_PATH_PKL)
print("✅ Admin ML model loaded")

# -------------------------------
# 4️⃣ Initialize DB
# -------------------------------
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
# Reports table
cursor.execute("""
CREATE TABLE IF NOT EXISTS reports (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    area TEXT,
    people_affected TEXT,
    citizen_message TEXT,
    flood_severity TEXT,
    flooded_area_ratio REAL,
    recommended_resources TEXT,
    report_status TEXT DEFAULT 'Pending',
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")
# Users table (role-based login)
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT,
    role TEXT CHECK(role IN ('admin','citizen','relief'))
)
""")
conn.commit()
conn.close()
print("✅ Database ready")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Insert default users (only if not exists)
users = [
    ("admin", "admin123", "admin"),
    ("citizen", "citizen123", "citizen"),
    ("relief", "relief123", "relief")
]

for u in users:
    try:
        cursor.execute(
            "INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
            u
        )
    except:
        pass  # user already exists

conn.commit()
conn.close()

print("✅ Default users created")

# -------------------------------
# 5️⃣ Flask App
# -------------------------------
app = Flask(__name__)
app.secret_key = "supersecretkey"
CORS(app)

# -------------------------------
# 6️⃣ Helper functions
# -------------------------------
def preprocess_image(img_bytes):
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    img = img.resize((256,256))
    img = np.array(img)/255.0
    return np.expand_dims(img, axis=0)

def analyze_mask(mask):
    flooded = np.sum(mask>0.5)
    ratio = flooded/mask.size
    if ratio>0.6:
        return "Severe", ratio, ["Boats","Food","Medical","Shelter"]
    elif ratio>0.3:
        return "Moderate", ratio, ["Food","Shelter"]
    return "Low", ratio, ["Monitoring"]

# -------------------------------
# 7️⃣ Routes
# -------------------------------
@app.route("/")
def index():
    return "🌊 AI Flood Relief Combined Backend Running"

# 7a. Login
@app.route("/login", methods=["POST", "OPTIONS"])
def login():
    if request.method == "OPTIONS":
        return ("", 204)

    data = request.get_json(force=True)
    username = data.get("username")
    password = data.get("password")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute(
        "SELECT role FROM users WHERE username=? AND password=?",
        (username, password)
    )
    row = cursor.fetchone()
    conn.close()

    if row:
        return jsonify({"success": True, "role": row[0]})

    return jsonify({"success": False, "message": "Invalid credentials"}), 401

# 7b. Citizen: submit report
@app.route("/report", methods=["POST"])
def submit_report():
    image = request.files["image"]
    text = request.form.get("text","")
    area = request.form.get("area","")
    people = request.form.get("people","")
    img = preprocess_image(image.read())
    pred = cnn_model.predict(img)
    mask = np.squeeze(pred)
    if mask.ndim==3: mask = mask.mean(axis=-1)
    severity, ratio, resources = analyze_mask(mask)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO reports (area, people_affected, citizen_message, flood_severity, flooded_area_ratio, recommended_resources)
        VALUES (?,?,?,?,?,?)
    """,(area, people, text, severity, float(ratio), ",".join(resources)))
    conn.commit(); conn.close()
    return jsonify({"flood_severity": severity,"flooded_area_ratio": round(float(ratio),2),"recommended_resources": resources})

# 7c. Admin/Relief APIs
@app.route("/reports", methods=["GET"])
def get_reports():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM reports ORDER BY timestamp DESC")
    rows = cursor.fetchall(); conn.close()
    keys = ["id","area","people_affected","citizen_message","flood_severity","flooded_area_ratio","recommended_resources","report_status","timestamp"]
    return jsonify([dict(zip(keys,r)) for r in rows])

@app.route("/reports/assigned", methods=["GET"])
def get_assigned_reports():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM reports WHERE UPPER(TRIM(report_status))='ASSIGNED' ORDER BY timestamp DESC")
    rows = cursor.fetchall(); conn.close()
    keys = ["id","area","people_affected","citizen_message","flood_severity","flooded_area_ratio","recommended_resources","report_status","timestamp"]
    return jsonify([dict(zip(keys,r)) for r in rows])

@app.route("/reports/update", methods=["POST"])
def update_report():
    data = request.get_json(force=True)
    report_id = data.get("id")
    status = data.get("status")
    if not report_id or not status: return jsonify({"success":False,"message":"Missing id/status"}),400
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("UPDATE reports SET report_status=? WHERE id=?",(status.strip(), report_id))
    conn.commit(); conn.close()
    return jsonify({"success":True,"message":"Report updated"})

# 7d. Admin: Predict flood (tabular)
@app.route("/predict-flood", methods=["POST"])
def predict_flood():
    data = request.json.get("features")
    if not data or len(data)!=20:
        return jsonify({"error":"Exactly 20 features required"}),400
    features = np.array(data).reshape(1,-1)
    prediction = ml_model.predict(features)[0]
    if prediction<0.3: risk="Low"
    elif prediction<0.6: risk="Moderate"
    else: risk="High"
    return jsonify({"flood_probability": round(float(prediction),3),"risk_level": risk})

# -------------------------------
# 8️⃣ Run server & ngrok
# -------------------------------
def run():
    app.run(host="0.0.0.0", port=5000, debug=False)

threading.Thread(target=run).start()
public_url = ngrok.connect(5000)
print("🌍 PUBLIC URL:", public_url)




Mounted at /content/drive
✅ CNN model loaded


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✅ Admin ML model loaded
✅ Database ready
✅ Default users created
 * Serving Flask app '__main__'
 * Debug mode: off


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


🌍 PUBLIC URL: NgrokTunnel: "https://a831-34-106-180-35.ngrok-free.app" -> "http://localhost:5000"
